## Feature Expansion — Geo + Seasonality
Goal: Add district/beat priors, lat/lon clusters, weekend/season/holiday flags. Then retrain HGB baseline and compare to 0.663 stack.

In [1]:
import os, time, json, tempfile
from pathlib import Path 
import numpy as np, pandas as pd 

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

from sklearn.metrics import (
    average_precision_score, roc_auc_score, precision_recall_curve, roc_curve,
    classification_report, confusion_matrix
)
from scipy.stats import randint, loguniform
from sklearn.ensemble import HistGradientBoostingClassifier

import matplotlib.pyplot as plt

# Paths
REPO = Path.cwd()
while REPO.name != "chicago-crime-pipeline" and REPO.parent != REPO:
    REPO = REPO.parent
DATA = REPO / "data" / "processed"
ART  = REPO / "notebooks" / "artifacts"
ART.mkdir(parents=True, exist_ok=True)

stamp = time.strftime("%Y%m%d-%H%M%S")
print("ART:", ART, "| stamp:", stamp)

ART: /Volumes/easystore/Projects/chicago-crime-pipeline/notebooks/artifacts | stamp: 20250922-234027


In [2]:
# Load and split 
df = pd.read_csv(DATA / "arrest_features.csv")
assert "arrest" in df.columns, df.columns

# (If these columns exist, great. If not, the geo/season steps will gracefully skip.)
print("shape:", df.shape)
print(df.columns.tolist()[:30])

TARGET = "arrest"
y = df[TARGET].astype(int).values
X = df.drop(columns=[TARGET]).copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

shape: (10482, 10)
['id', 'date', 'primary_type', 'location_description', 'arrest', 'location_grouped', 'year', 'month', 'dow', 'hour']


In [3]:
# Make editable copies
X_train_fe = X_train.copy()
X_test_fe  = X_test.copy()

# Weekday from date
for Xdf in (X_train_fe, X_test_fe):
    Xdf["weekday"] = pd.to_datetime(Xdf["date"]).dt.day_name()

# Hour bins
bins, labels = [0,6,12,18,24], ["00-05","06-11","12-17","18-23"]
for Xdf in (X_train_fe, X_test_fe):
    Xdf["hour_bin"] = pd.cut(Xdf["hour"].astype(int), bins=bins, right=False, labels=labels)

# Rare bucketing helper
def rare_bucket(train_col, test_col, min_count=40):
    vc = train_col.value_counts()
    keep = set(vc[vc >= min_count].index)
    return (train_col.where(train_col.isin(keep), "__RARE__"),
            test_col.where(test_col.isin(keep),  "__RARE__"))

for col in ["location_description","primary_type"]:
    X_train_fe[col], X_test_fe[col] = rare_bucket(X_train_fe[col], X_test_fe[col], 40)

# Frequency encodings
def add_freq(col):
    freq = X_train_fe[col].astype(object).value_counts(normalize=True)
    X_train_fe[f"{col}_freq"] = X_train_fe[col].map(freq).astype(float).fillna(0.0)
    X_test_fe[f"{col}_freq"]  = X_test_fe[col].map(freq).astype(float).fillna(0.0)

for c in ["primary_type","location_description","weekday","hour_bin"]:
    add_freq(c)

# Target mean for primary_type (prior)
ptype_rate = pd.Series(y_train).groupby(X_train_fe["primary_type"]).mean()
X_train_fe["ptype_arrest_rate"] = X_train_fe["primary_type"].map(ptype_rate)
X_test_fe["ptype_arrest_rate"]  = X_test_fe["primary_type"].map(ptype_rate).fillna(float(ptype_rate.mean()))

# Interaction ptype x hour_bin
X_train_fe["ptype_x_hourbin"] = X_train_fe["primary_type"].astype(str) + "_" + X_train_fe["hour_bin"].astype(str)
X_test_fe["ptype_x_hourbin"]  = X_test_fe["primary_type"].astype(str)  + "_" + X_test_fe["hour_bin"].astype(str)

# Cyclical time
for Xdf in (X_train_fe, X_test_fe):
    Xdf["hour_sin"]  = np.sin(2*np.pi * Xdf["hour"].astype(float)/24.0)
    Xdf["hour_cos"]  = np.cos(2*np.pi * Xdf["hour"].astype(float)/24.0)
    Xdf["month_sin"] = np.sin(2*np.pi * Xdf["month"].astype(float)/12.0)
    Xdf["month_cos"] = np.cos(2*np.pi * Xdf["month"].astype(float)/12.0)

In [4]:
# (A) District/beat priors if present 
geo_cols_present = set(X_train_fe.columns)

if "district" in geo_cols_present:
    dist_rate = pd.Series(y_train).groupby(X_train_fe["district"]).mean()
    X_train_fe["district_arrest_rate"] = X_train_fe["district"].map(dist_rate)
    X_test_fe["district_arrest_rate"]  = X_test_fe["district"].map(dist_rate).fillna(float(dist_rate.mean()))
else:
    print("[note] 'district' not found; skipping district priors.")

if "beat" in geo_cols_present:
    beat_rate = pd.Series(y_train).groupby(X_train_fe["beat"]).mean()
    X_train_fe["beat_arrest_rate"] = X_train_fe["beat"].map(beat_rate)
    X_test_fe["beat_arrest_rate"]  = X_test_fe["beat"].map(beat_rate).fillna(float(beat_rate.mean()))
else:
    print("[note] 'beat' not found; skipping beat priors.")

# (B) Lat/Lon clustering → spatial buckets (if present)
if {"latitude","longitude"}.issubset(geo_cols_present):
    # Use KMeans on train; same transform on test
    k = 25  # tune later (e.g., 20–60)
    kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto")
    coords_train = X_train_fe[["latitude","longitude"]].astype(float).values
    coords_test  = X_test_fe[["latitude","longitude"]].astype(float).values
    X_train_fe["geo_cluster"] = kmeans.fit_predict(coords_train).astype(str)
    X_test_fe["geo_cluster"]  = kmeans.predict(coords_test).astype(str)

    # cluster priors
    gc_rate = pd.Series(y_train).groupby(X_train_fe["geo_cluster"]).mean()
    X_train_fe["geo_cluster_rate"] = X_train_fe["geo_cluster"].map(gc_rate)
    X_test_fe["geo_cluster_rate"]  = X_test_fe["geo_cluster"].map(gc_rate).fillna(float(gc_rate.mean()))
else:
    print("[note] latitude/longitude not found; skipping KMeans geo_cluster.")

[note] 'district' not found; skipping district priors.
[note] 'beat' not found; skipping beat priors.
[note] latitude/longitude not found; skipping KMeans geo_cluster.


In [5]:
# New seasonal features

# Weekend
for Xdf in (X_train_fe, X_test_fe):
    Xdf["is_weekend"] = pd.to_datetime(Xdf["date"]).dt.dayofweek.isin([5,6]).astype(int)

# Summer (Jun-Aug)
for Xdf in (X_train_fe, X_test_fe):
    Xdf["is_summer"] = Xdf["month"].isin([6,7,8]).astype(int)

# US Federal Holidays (approx; if date has no year, parse as datetime)
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
years = sorted(set(pd.to_datetime(X_train_fe["date"]).dt.year) | set(pd.to_datetime(X_test_fe["date"]).dt.year))
holidays = cal.holidays(start=f"{min(years)}-01-01", end=f"{max(years)}-12-31")
holidays = set(pd.to_datetime(holidays).date)

def is_holiday(s):
    d = pd.to_datetime(s).dt.date
    return d.isin(holidays).astype(int)

X_train_fe["is_holiday"] = is_holiday(X_train_fe["date"])
X_test_fe["is_holiday"]  = is_holiday(X_test_fe["date"])

In [ ]:
# Preprocessor builder

cat_base = [
    "date","primary_type","location_description","location_grouped",
    "weekday","hour_bin","ptype_x_hourbin",
    # geo buckets if exist
    "district","beat","geo_cluster"
]
num_base = [
    "id","year","month","dow","hour",
    "primary_type_freq","location_description_freq","weekday_freq","hour_bin_freq",
    "ptype_arrest_rate",
    # geo priors if exist
    "district_arrest_rate","beat_arrest_rate","geo_cluster_rate",
    # cyclical + seasonal
    "hour_sin","hour_cos","month_sin","month_cos",
    "is_weekend","is_summer","is_holiday",
    # raw coords if you want (can help trees a bit)
    "latitude","longitude"
]

def build_preprocessor(Xdf):
    present = set(Xdf.columns)
    cat_cols = [c for c in cat_base if c in present]
    num_cols = [c for c in num_base if c in present]
    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
            ("num", "passthrough", num_cols),
        ],
        remainder="drop",
        verbose_feature_names_out=False,
    )
    return pre, cat_cols, num_cols

pre_exp, cat_cols_exp, num_cols_exp = build_preprocessor(X_train_fe)
print("CATS:", cat_cols_exp)
print("NUMS:", num_cols_exp)

CATS: ['date', 'primary_type', 'location_description', 'location_grouped', 'weekday', 'hour_bin', 'ptype_x_hourbin']
NUMS: ['id', 'year', 'month', 'dow', 'hour', 'primary_type_freq', 'location_description_freq', 'weekday_freq', 'hour_bin_freq', 'ptype_arrest_rate', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'is_weekend', 'is_summer', 'is_holiday']


In [7]:
# Quick HGB baseline on expanded features

pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()
sw_train = np.where(y_train==1, pos_weight, 1.0)

hgb = HistGradientBoostingClassifier(
    random_state=42, max_bins=255
)

param_dist = {
    "clf__learning_rate": loguniform(0.03, 0.2),
    "clf__max_depth": randint(4, 9),
    "clf__max_leaf_nodes": randint(24, 96),
    "clf__min_samples_leaf": randint(40, 160),
    "clf__l2_regularization": loguniform(1e-4, 5.0),
    "clf__max_iter": randint(200, 500),
}

pipe = Pipeline([("pre", pre_exp), ("clf", hgb)], memory=tempfile.mkdtemp())
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# quick n_iter so it finishes fast; bump to 20–30 later
search = RandomizedSearchCV(
    pipe, param_distributions=param_dist, n_iter=12,
    scoring="average_precision", refit=True, cv=cv, n_jobs=-1, random_state=42, verbose=2
)
search.fit(X_train_fe, y_train, clf__sample_weight=sw_train)
print("Best params:", search.best_params_, "| CV AP:", round(search.best_score_, 4))

final = search.best_estimator_.fit(X_train_fe, y_train, clf__sample_weight=sw_train)
proba = final.predict_proba(X_test_fe)[:,1]

ap  = average_precision_score(y_test, proba)
roc = roc_auc_score(y_test, proba)

prec, rec, thr = precision_recall_curve(y_test, proba)
f1s = 2*prec*rec/(prec+rec+1e-12)
i = np.nanargmax(f1s)
thr_best = thr[i] if i < len(thr) else 0.5
pred = (proba >= thr_best).astype(int)

print("\n=== TEST metrics (HGB + GEO/SEASON) ===")
print("PR-AUC:", round(ap, 4))
print("ROC-AUC:", round(roc, 4))
print("Best threshold:", float(thr_best), "Best F1:", float(f1s[i]))
print(classification_report(y_test, pred, digits=3))
print("Confusion:\n", confusion_matrix(y_test, pred))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END clf__l2_regularization=0.0057537737941171915, clf__learning_rate=0.18214744423753768, clf__max_depth=6, clf__max_iter=271, clf__max_leaf_nodes=84, clf__min_samples_leaf=60; total time= 1.2min
[CV] END clf__l2_regularization=0.0057537737941171915, clf__learning_rate=0.18214744423753768, clf__max_depth=6, clf__max_iter=271, clf__max_leaf_nodes=84, clf__min_samples_leaf=60; total time= 1.2min
[CV] END clf__l2_regularization=0.0057537737941171915, clf__learning_rate=0.18214744423753768, clf__max_depth=6, clf__max_iter=271, clf__max_leaf_nodes=84, clf__min_samples_leaf=60; total time= 1.2min
[CV] END clf__l2_regularization=0.0005409123677836541, clf__learning_rate=0.0403316977206044, clf__max_depth=6, clf__max_iter=287, clf__max_leaf_nodes=47, clf__min_samples_leaf=42; total time= 1.4min
[CV] END clf__l2_regularization=0.0005409123677836541, clf__learning_rate=0.0403316977206044, clf__max_depth=6, clf__max_iter=287, clf__

In [9]:
meta = {
    "timestamp": stamp,
    "approach": "HGB + geo/season expansion",
    "test_pr_auc": float(ap),
    "test_roc_auc": float(roc),
    "best_threshold": float(thr_best),
    "best_params": {k: (float(v) if hasattr(v, "item") else v) for k,v in search.best_params_.items()}
}
with open(ART / f"hgb_geo_season_meta_{stamp}.json", "w") as f:
    json.dump(meta, f, indent=2)

prec_, rec_, _ = precision_recall_curve(y_test, proba)
fpr, tpr, _ = roc_curve(y_test, proba)

plt.figure(); plt.plot(rec_, prec_); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.grid(True, alpha=0.3)
plt.title(f"HGB + GEO/SEASON PR (AP={ap:.3f})")
plt.savefig(ART / f"pr_curve_hgb_geo_{stamp}.png", bbox_inches="tight"); plt.close()

plt.figure(); plt.plot(fpr, tpr); plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.grid(True, alpha=0.3)
plt.title(f"HGB + GEO/SEASON ROC (AUC={roc:.3f})")
plt.savefig(ART / f"roc_curve_hgb_geo_{stamp}.png", bbox_inches="tight"); plt.close()

with open(ART / f"decision_threshold_hgb_geo_{stamp}.txt", "w") as f:
    f.write(str(thr_best))

print("Saved artifacts →", ART)

Saved artifacts → /Volumes/easystore/Projects/chicago-crime-pipeline/notebooks/artifacts


In [10]:
def slice_metrics(X_df, y_true, proba, threshold, slice_col, min_support=40):
    if slice_col not in X_df.columns: return None
    from sklearn.metrics import precision_recall_fscore_support
    d = pd.DataFrame({slice_col: X_df[slice_col], "y": y_true, "pred": (proba >= threshold).astype(int)})
    rows=[]
    for val,g in d.groupby(slice_col):
        if len(g)<min_support: continue
        p,r,f1,_ = precision_recall_fscore_support(g["y"], g["pred"], average="binary", zero_division=0)
        rows.append({slice_col: val, "support": int(len(g)), "precision": float(p), "recall": float(r), "f1": float(f1)})
    return pd.DataFrame(rows).sort_values("f1", ascending=False) if rows else None

for col in ["primary_type","weekday","hour_bin","district","beat","geo_cluster"]:
    tbl = slice_metrics(X_test_fe, y_test, proba, thr_best, col, min_support=40)
    if tbl is not None:
        out = ART / f"slice_metrics_{col}_hgb_geo_{stamp}.csv"
        tbl.to_csv(out, index=False); print("Saved:", out)

Saved: /Volumes/easystore/Projects/chicago-crime-pipeline/notebooks/artifacts/slice_metrics_primary_type_hgb_geo_20250922-234027.csv
Saved: /Volumes/easystore/Projects/chicago-crime-pipeline/notebooks/artifacts/slice_metrics_weekday_hgb_geo_20250922-234027.csv
Saved: /Volumes/easystore/Projects/chicago-crime-pipeline/notebooks/artifacts/slice_metrics_hour_bin_hgb_geo_20250922-234027.csv


/var/folders/6z/l9wv3crd4n5bzgcrdd7vxq8m0000gn/T/ipykernel_14455/535738392.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for val,g in d.groupby(slice_col):


In [11]:
from sklearn.ensemble import RandomForestClassifier

rf_geo = Pipeline(steps=[
    ("pre", pre_exp),
    ("clf", RandomForestClassifier(
        n_estimators=300,
        max_depth=12,
        min_samples_split=6,
        min_samples_leaf=6,
        max_features=0.6,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42,
        bootstrap=True
    ))
])

rf_geo.fit(X_train_fe, y_train)
proba_rf_geo = rf_geo.predict_proba(X_test_fe)[:,1]
ap_rf_geo = average_precision_score(y_test, proba_rf_geo)
roc_rf_geo = roc_auc_score(y_test, proba_rf_geo)
print("RF (geo/season) PR-AUC:", round(ap_rf_geo,4), "| ROC-AUC:", round(roc_rf_geo,4))

RF (geo/season) PR-AUC: 0.6479 | ROC-AUC: 0.8739


In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
oof_rf = np.zeros(len(y_train)); oof_hgb = np.zeros(len(y_train))

for fold,(tr,val) in enumerate(cv.split(X_train_fe, y_train),1):
    Xtr, Xval = X_train_fe.iloc[tr], X_train_fe.iloc[val]
    ytr, yval = y_train[tr], y_train[val]

    rf_f  = Pipeline([("pre", pre_exp), ("clf", rf_geo.named_steps["clf"].__class__(**rf_geo.named_steps["clf"].get_params()))])
    hgb_f = Pipeline([("pre", pre_exp), ("clf", final.named_steps["clf"].__class__(**final.named_steps["clf"].get_params()))])

    rf_f.fit(Xtr, ytr)
    hgb_f.fit(Xtr, ytr)

    oof_rf[val]  = rf_f.predict_proba(Xval)[:,1]
    oof_hgb[val] = hgb_f.predict_proba(Xval)[:,1]

X_meta_geo = np.column_stack([oof_rf, oof_hgb])
meta_geo = LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42)
meta_geo.fit(X_meta_geo, y_train)

# fit full bases on all train
rf_full_geo  = Pipeline([("pre", pre_exp), ("clf", rf_geo.named_steps["clf"].__class__(**rf_geo.named_steps["clf"].get_params()))]).fit(X_train_fe, y_train)
hgb_full_geo = Pipeline([("pre", pre_exp), ("clf", final.named_steps["clf"].__class__(**final.named_steps["clf"].get_params()))]).fit(X_train_fe, y_train)

# test stack
rf_test_geo  = rf_full_geo.predict_proba(X_test_fe)[:,1]
hgb_test_geo = hgb_full_geo.predict_proba(X_test_fe)[:,1]
proba_stack_geo = meta_geo.predict_proba(np.column_stack([rf_test_geo, hgb_test_geo]))[:,1]

ap_stack  = average_precision_score(y_test, proba_stack_geo)
roc_stack = roc_auc_score(y_test, proba_stack_geo)

prec, rec, thr = precision_recall_curve(y_test, proba_stack_geo)
f1s = 2*prec*rec/(prec+rec+1e-12)
i = np.nanargmax(f1s)
thr_stack_geo = thr[i] if i < len(thr) else 0.5
pred_stack_geo = (proba_stack_geo >= thr_stack_geo).astype(int)

print("\n=== TEST metrics (STACK RF+HGB on geo/season) ===")
print("PR-AUC:", round(ap_stack,4))
print("ROC-AUC:", round(roc_stack,4))
print("Best threshold:", float(thr_stack_geo), "Best F1:", float(f1s[i]))
print(classification_report(y_test, pred_stack_geo, digits=3))
print("Confusion:\n", confusion_matrix(y_test, pred_stack_geo))


=== TEST metrics (STACK RF+HGB on geo/season) ===
PR-AUC: 0.6601
ROC-AUC: 0.881
Best threshold: 0.7695998051555637 Best F1: 0.6204379562038848
              precision    recall  f1-score   support

           0      0.929     0.958     0.943      1795
           1      0.691     0.563     0.620       302

    accuracy                          0.901      2097
   macro avg      0.810     0.760     0.782      2097
weighted avg      0.894     0.901     0.897      2097

Confusion:
 [[1719   76]
 [ 132  170]]


In [13]:
import joblib

stack_geo_meta = {
    "timestamp": stamp,
    "approach": "STACK RF+HGB on geo/season features",
    "test_pr_auc": float(ap_stack),
    "test_roc_auc": float(roc_stack),
    "best_threshold": float(thr_stack_geo),
}
with open(ART / f"stack_geo_meta_{stamp}.json", "w") as f:
    json.dump(stack_geo_meta, f, indent=2)

with open(ART / f"decision_threshold_stack_geo_{stamp}.txt", "w") as f:
    f.write(str(thr_stack_geo))

# Save full objects so you can load in prod
joblib.dump({"rf": rf_full_geo, "hgb": hgb_full_geo, "meta": meta_geo}, ART / f"stack_geo_models_{stamp}.pkl")
print("Saved stack models →", ART / f"stack_geo_models_{stamp}.pkl")

Saved stack models → /Volumes/easystore/Projects/chicago-crime-pipeline/notebooks/artifacts/stack_geo_models_20250922-234027.pkl
